In [2]:
import nltk
# nltk.download()

In [3]:
from xml.dom import minidom
import pandas as pd
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import unicodedata
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn import svm
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

In [3]:
mydoc = minidom.parse('data/train.xml')

In [4]:
commentaires = mydoc.getElementsByTagName("commentaire")
notes = mydoc.getElementsByTagName("note")

In [5]:
row_list = []

for i in range(len(notes)):
#     if i%20000==0: print(i)
    try:
        row = {}
        row['note'] = notes[i].firstChild.nodeValue
        row['commentaire'] = commentaires[i].firstChild.nodeValue
        row_list.append(row)
    except Exception:
        print("line error",i)
    
train = pd.DataFrame(row_list)


line error 40959
line error 138550
line error 267600
line error 280534
line error 307511
line error 342799
line error 380125
line error 528530
line error 541831
line error 543469
line error 621082
line error 663582


In [16]:
punct_list = re.sub("[?!'.,]",'',string.punctuation)
punct_list += '«»“”‘'+chr(9)+chr(10)+chr(13)
punct_list

'"#$%&()*+-/:;<=>@[\\]^_`{|}~«»“”‘\t\n\r'

In [17]:
def remove_punct(text):    
    text_nopunct = "".join([char for char in text if char not in punct_list])# It will discard all punctuations
    return text_nopunct



In [18]:
# # Function to Tokenize words
def tokenize(text):    
    #Tokenize les series de marque de ponctuation, reduit les espaces, supprime les tokens vides
    text = unicodedata.normalize('NFKD',text)
    whitespace_index = []
    for ite in re.finditer("!+|\?+", text, flags=0):
        whitespace_index.append(ite.start(0))
        whitespace_index.append(ite.end(0))

    for i in reversed(whitespace_index):
        text = text[:i]+" "+text[i:]
    text_clean = re.sub("^ | $","",re.sub(" +"," ",re.sub("[,.']+"," ",text)))
    if len(text_clean) == 0:
        return []
    tokens = re.split("[\s]",text_clean)
    return tokens



In [19]:
stopword = nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('french')
# stopword

In [20]:
# Function to remove Stopwords
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]# To remove all stopwords
    return text

In [21]:
sno = nltk.stem.SnowballStemmer('french')

def stemming(tokenized_text):
    text = [sno.stem(word) for word in tokenized_text]
    return text

In [22]:
wn = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

In [25]:
def clean_text(text):
    return stemming(remove_stopwords(tokenize(remove_punct(text).lower())))

In [13]:
train['commentaire_punct_clean'] = train['commentaire'].apply(lambda x: remove_punct(x))
train['commentaire_tokenized'] = train['commentaire_punct_clean'].apply(lambda x: tokenize(x.lower())) 
train['commentaire_nostop'] = train['commentaire_tokenized'].apply(lambda x: remove_stopwords(x))
train['commentaire_stemmed'] = train['commentaire_nostop'].apply(lambda x: stemming(x))
train['commentaire_lemmatized'] = train['commentaire_nostop'].apply(lambda x: lemmatizing(x))

In [14]:
train.head()

,note,commentaire,commentaire_punct_clean,commentaire_tokenized,commentaire_nostop,commentaire_stemmed,commentaire_lemmatized
0,"4,0","""Ce n'est pas ce qui se trouve en face de vous...","Ce n'est pas ce qui se trouve en face de vous,...","[ce, n, est, pas, ce, qui, se, trouve, en, fac...","[trouve, face, tient, à, côté, cette, phras...","[trouv, fac, tient, à, côté, cet, phras, ta...","[trouve, face, tient, à, côté, cette, phras..."
1,"3,5","(...) En soi, je n'ai rien contre ce genre de ...","... En soi, je n'ai rien contre ce genre de ré...","[en, soi, je, n, ai, rien, contre, ce, genre, ...","[soi, rien, contre, genre, récit, hyper, codi...","[soi, rien, contr, genr, réc, hyp, codifié, ...","[soi, rien, contre, genre, récit, hyper, codi..."
2,"4,5","Pour sa troisième réalisation après ""Tron, l'h...","Pour sa troisième réalisation après Tron, l'hé...","[pour, sa, troisième, réalisation, après, t...","[troisième, réalisation, après, tron, héri...","[troisièm, réalis, aprè, tron, héritag, ob...","[troisième, réalisation, après, tron, héri..."
3,"3,0",Joseph Kosinski offre de nouvelles images ébou...,Joseph Kosinski offre de nouvelles images ébou...,"[joseph, kosinski, offre, de, nouvelles, image...","[joseph, kosinski, offre, nouvelles, images, e...","[joseph, kosinsk, offre, nouvel, imag, ébouri...","[joseph, kosinski, offre, nouvelles, image, é..."
4,"3,5",J'ai vu un film... qui traite d'un drame humai...,J'ai vu un film... qui traite d'un drame humai...,"[j, ai, vu, un, film, qui, traite, d, un, dram...","[vu, film, traite, drame, humain, plus, haute,...","[vu, film, trait, dram, humain, plus, haut, in...","[vu, film, traite, drame, humain, plus, haute,..."


In [15]:
# CSV(SemiColon-separated-value)
train[['note','commentaire_stemmed']].to_csv("train_clean_stem.csv", sep=';')
train[['note','commentaire_lemmatized']].to_csv("train_clean_lem.csv", sep=';')
#train[['note','commentaire']].to_csv("train_clean.csv", sep=';')

In [14]:
#train = pd.read_csv("train_clean.csv", sep=';',index_col=0)


In [15]:
ttest = "??Ce n'e,,,st pa..,.s ce!! qu...     i se.tro???uve en face de!!!!??"
ttest = remove_punct(ttest.lower())
print(ttest)
ttest = tokenize(ttest)

ttest

??ce n'e,,,st pa..,.s ce!! qu...     i se.tro???uve en face de!!!!??


['??',
 'ce',
 'n',
 'e',
 'st',
 'pa',
 's',
 'ce',
 '!!',
 'qu',
 'i',
 'se',
 'tro',
 '???',
 'uve',
 'en',
 'face',
 'de',
 '!!!!',
 '??']

In [17]:
# ngram_vect = CountVectorizer(ngram_range=(2,2),analyzer=clean_text) # It applies only bigram vectorizer
# X_counts = ngram_vect.fit_transform(train['commentaire'])
# X_counts_df = pd.DataFrame(X_counts.toarray(), columns=ngram_vect.get_feature_names())


tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(train['commentaire'])
#X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names())

In [19]:
labelfloat = train['note'].apply(lambda x: re.sub(",",'.',x))
# labelfloat = train['note'].apply(lambda x: re.sub(",",'.',x))
notes = {0.5: 0,1.0: 1,1.5: 2,2.0: 3,2.5: 4,3.0: 5,3.5: 6,4.0: 7,4.5: 8,5.0: 9}
noteINT = labelfloat.apply(lambda x: notes[float(x)])
noteINT

0         7
1         6
2         8
3         5
4         6
         ..
665945    7
665946    5
665947    6
665948    5
665949    6
Name: note, Length: 665950, dtype: int64

In [20]:
svd = TruncatedSVD(100)
X_tfidf_reduced = svd.fit_transform(X_tfidf)

In [21]:
#X_tfidf_reduced.to_csv("X_tfidf_reduced.csv", sep=';')

In [22]:
x_train, x_test, y_train, y_test = train_test_split(X_tfidf_reduced,noteINT, test_size=0.2, random_state=1)
clf = LinearSVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
#clf.score(x_test,y_test)

#print(clf.predict(X_tfidf_reduced[10].reshape(1,-1))[0])

In [23]:
x_test

array([[ 0.28197623, -0.01102014, -0.07466781, ...,  0.00298113,
         0.02295452,  0.01222901],
       [ 0.20072312, -0.06687433,  0.02126142, ...,  0.00876331,
        -0.02653064,  0.02024149],
       [ 0.24677846,  0.01265841,  0.09127958, ..., -0.00046889,
         0.0307302 , -0.01408136],
       ...,
       [ 0.02001199,  0.01413156, -0.02031568, ..., -0.00076455,
        -0.00250706,  0.00261329],
       [ 0.09925962, -0.05297585,  0.01445042, ...,  0.0430905 ,
         0.02407453, -0.02110756],
       [ 0.0948539 ,  0.00459399, -0.01241713, ...,  0.06926825,
         0.03563151, -0.01739512]])

In [29]:
testxml = minidom.parse('data/test.xml')

In [30]:
commentaires = testxml.getElementsByTagName("commentaire")
review_ids = testxml.getElementsByTagName("review_id")
#notes = testxml.getElementsByTagName("note")
row_list = []
for i in range(len(commentaires)):
#     if i%20000==0: print(i)
    try:
        row = {}
        #row['note'] = notes[i].firstChild.nodeValue
        row['review_id'] = review_ids[i].firstChild.nodeValue
        row['commentaire'] = commentaires[i].firstChild.nodeValue
        row_list.append(row)
    except Exception:
        print("line error",i)
    
test = pd.DataFrame(row_list)

In [31]:
test.head()

,review_id,commentaire
0,review_59354742,"12 jours, un film que j'ai raté à sa sortie et..."
1,review_62570109,Ces lueurs dans ces yeux... Soit ils sont tous...
2,review_58180650,Il s’agit d’un documentaire sans la moindre ac...
3,review_57696986,un documentaire magnifique dans cette unité ly...
4,review_57736972,"Un documentaire très intéressant, quand bien m..."


In [32]:
test.head()
tfidf_test_vect = TfidfVectorizer(analyzer=clean_text)
X_test_tfidf = tfidf_test_vect.fit_transform(test['commentaire'])
#svd = TruncatedSVD(100)
X_test_tfidf_reduced = svd.fit_transform(X_test_tfidf)

In [33]:
y_pred_test = clf.predict(X_test_tfidf_reduced)
f = open("results.txt", "a")
print("review , prediction")
notes_virgules = {"0,5": 0,"1,0": 1,"1,5": 2,"2,0": 3,"2,5": 4,"3,0": 5,"3,5": 6,"4,0": 7,"4,5": 8,"5,0": 9}

key_list = list(notes_virgules.keys())
val_list = list(notes_virgules.values())
for i in range(len(y_pred_test)):
    position = val_list.index(y_pred_test[i])
    n = key_list[position]
    f.write(test.iloc[i]['review_id']+" "+str(n)+"\n")
    #print(test.iloc[i]['review_id']," ",str(n),"\n")
f.close()

review , prediction


In [34]:
#test.iloc[[0]]['review_id'][0]
print(y_pred_test[0])


print(key_list)

7
['0,5', '1,0', '1,5', '2,0', '2,5', '3,0', '3,5', '4,0', '4,5', '5,0']


In [35]:
my_dict ={"java":100, "python":112, "c":11}
 
# list out keys and values separately
key_list = list(notes.keys())
val_list = list(notes.values())
 
# print key with val 100
position = val_list.index(y_pred_test[0])
print(key_list[position])
 
# print key with val 112
position = val_list.index(2)
print(key_list[position])

4.0
1.5


In [27]:
mydoc = minidom.parse('data/dev.xml')
commentaires = mydoc.getElementsByTagName("commentaire")
notes = mydoc.getElementsByTagName("note")
row_list = []

for i in range(len(notes)):
#     if i%20000==0: print(i)
    try:
        row = {}
        row['note'] = notes[i].firstChild.nodeValue
        row['commentaire'] = commentaires[i].firstChild.nodeValue
        row_list.append(row)
    except Exception:
        print("line error",i)
    
dev = pd.DataFrame(row_list)


line error 46611
line error 66228


In [28]:
dev['commentaire_stemmed'] = dev['commentaire'].apply(lambda x: clean_text(x))
dev[['note','commentaire_stemmed']].to_csv("dev_clean_stem.csv", sep=';')

In [32]:
mydoc = minidom.parse('data/test.xml')
commentaires = mydoc.getElementsByTagName("commentaire")
review_ids = mydoc.getElementsByTagName("review_id")
row_list = []
for i in range(len(commentaires)):
#     if i%20000==0: print(i)
    try:
        row = {}
        #row['note'] = notes[i].firstChild.nodeValue
        row['review_id'] = review_ids[i].firstChild.nodeValue
        row['commentaire'] = commentaires[i].firstChild.nodeValue
        row_list.append(row)
    except Exception:
        print("line error",i)
    
test = pd.DataFrame(row_list)

In [33]:
test['commentaire_stemmed'] = test['commentaire'].apply(lambda x: clean_text(x))
test[['review_id','commentaire_stemmed']].to_csv("test_clean_stem.csv", sep=';')